In [27]:
import pandas as pd
from variables import app_vars
from sqlalchemy import create_engine
import json

%load_ext dotenv
%dotenv

connection_string = %env YANZI_CONNECTION_STRING
engine = create_engine(connection_string)
store_name = app_vars.STORE_NAME

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [28]:
sql = f"select * from transactions where Store = '{store_name}'"
pd.read_sql(sql, engine).to_csv(f'./assets/{store_name}_transactions.csv', index=False)

In [29]:
sql = f"select * from sync_info where store = '{store_name}' and SyncEntityType = 'Transaction'"
pd.read_sql(sql, engine).to_csv(f'./assets/{store_name}_transactions_sync_info.csv', index=False)

In [30]:
sql = (""
       "select td.* "
       "from transaction_discounts td "
       "inner join transactions t on t.id = td.TransactionId "
       f"where t.Store = '{store_name}' and t.DeleteById is null and td.DeleteById is null"
       "")

pd.read_sql(sql, engine).to_csv(f'./assets/{store_name}_transaction_discounts.csv', index=False)

In [31]:
sql = f"select * from store_configurations where StoreName = '{store_name}'"
data = json.loads(pd.read_sql(sql, engine).to_json(orient='records'))
app_vars.header = data[0]['Header']
app_vars.footer = data[0]['Footer']

In [32]:
engine.dispose(close=True)